<a href="https://colab.research.google.com/github/Tom-Jung/IO_with_python/blob/main/Ch4.%20%EC%82%B0%EC%97%85%EC%97%B0%EA%B4%80%ED%91%9C%EB%A5%BC%20%ED%86%B5%ED%95%9C%20%EA%B2%BD%EC%A0%9C%EA%B5%AC%EC%A1%B0%20%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np   # Numpy import
import pandas as pd  # Pandas import

###  3부문 총, 국산, 수입거래표 불러오기

In [36]:
url_Tt_3 = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Total_matrix_3sectors.csv"  # 3부문의 총거래표 불러오기
Tt_3 = pd.read_csv(url_Tt_3)

url_Dt_3 = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Domestic_matrix_3sectors.csv"  # 3부문의 국산거래표 불러오기
Dt_3 = pd.read_csv(url_Dt_3)

url_It_3 = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Imported_matrix_3sectors.csv"  # 3부문의 수입거래표 불러오기
It_3 = pd.read_csv(url_It_3)

url_emp_3 = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Emp_3_secs.csv"  # 3부문의 고용표 불러오기
Emp_3 = pd.read_csv(url_emp_3)

#### 산업구조_부가가치 및 산출액 비중 구하기

In [37]:
Tt = Tt_3.to_numpy()  # numpy로 불러오기
Dt = Dt_3.to_numpy()
It = It_3.to_numpy()
Em = Emp_3.to_numpy()

In [38]:
va = Tt[-2,1:4].astype(float)  # 산업별 부가가치
ct = Tt[-1,1:4].astype(float)  # 산업별 산출액

In [39]:
va_wgt = (va/va.sum()).reshape([3,1])  # 부가가치 기준 각 산업의 비중
ct_wgt = (ct/ct.sum()).reshape([3,1])  # 산출액 기준 각 산업의 비중

wgt_mx = np.concatenate((va_wgt, ct_wgt), axis=1).round(3)
print(wgt_mx)

[[0.017 0.014]
 [0.287 0.417]
 [0.696 0.569]]


#### 투입구조

In [40]:
Tt_itr = Tt[-3,1:5] # 총거래표의 각 산업별 및 총산업의 중간투입계
Dt_itr = Dt[-1,1:5] # 국산거래표의 각 산업별 및 총산업의 중간투입계
It_itr = It[-1,1:5] # 수입거래표의 각 산업별 및 총산업의 중간투입계
Ct_itr = Tt[-1,1:5] # 각 산업별 및 총산업의 산출액

print(Tt_itr)
print(Dt_itr)
print(It_itr)
print(Ct_itr)

[27.0 1239.0 1105.0 2371.0]
[26.0 862.0 976.0 1864.0]
[1.0 377.0 129.0 507.0]
[60.0 1792.0 2447.0 4299.0]


In [41]:
Tt_itr_ratio = (Tt_itr/Ct_itr).reshape([4,1]).astype(float) # 총투입계수
Dt_itr_ratio = (Dt_itr/Ct_itr).reshape([4,1]).astype(float) # 국산투입계수
It_itr_ratio = (It_itr/Ct_itr).reshape([4,1]).astype(float) # 수입투입계수

Itr_ratio = (np.concatenate((Tt_itr_ratio, Dt_itr_ratio, It_itr_ratio), axis=1)).round(3)  # 총, 국산, 수입투입계수
print(Itr_ratio)

[[0.45  0.433 0.017]
 [0.691 0.481 0.21 ]
 [0.452 0.399 0.053]
 [0.552 0.434 0.118]]


In [42]:
Dt_dm_itr = (Dt_itr/Tt_itr).reshape([4,1]).astype(float).round(3)  # 각 산업별, 총산업의 국산화율
Dt_dm_itr

array([[0.963],
       [0.696],
       [0.883],
       [0.786]])

In [43]:
Va_ratio = (Tt[-2,1:5]/Tt[-1,1:5]).reshape([4,1]).astype(float).round(3)  # 부가가치율 = 부가가치(3산업과 총부가가치)/산출액(3산업과 총산출액)
Va_ratio

array([[0.55 ],
       [0.309],
       [0.548],
       [0.448]])

In [44]:
#중간투입, 국산화율 및 부가가치율을 열방향으로 부치기
Input_str = ((np.concatenate((Itr_ratio, Dt_dm_itr, Va_ratio), axis=1))*100)
Input_str_f = Input_str.astype(float).round(1)
print(Input_str_f)

[[45.  43.3  1.7 96.3 55. ]
 [69.1 48.1 21.  69.6 30.9]
 [45.2 39.9  5.3 88.3 54.8]
 [55.2 43.4 11.8 78.6 44.8]]


In [45]:
Input_f = pd.DataFrame(Input_str_f, columns = ['총중간투입률', '국산중간투입률', '수입중간투입률', '국산화율', '부가가치율'],
                       index = ['농산품','공산품','서비스','전품목'])
Input_f

,총중간투입률,국산중간투입률,수입중간투입률,국산화율,부가가치율
농산품,45.0,43.3,1.7,96.3,55.0
공산품,69.1,48.1,21.0,69.6,30.9
서비스,45.2,39.9,5.3,88.3,54.8
전품목,55.2,43.4,11.8,78.6,44.8


#### 배분구조

In [46]:
Tot_sup =Tt[0:4,-1]  # 총공급(산출액+수입)
Dom_ct =Dt[0:4,-1]  # 산출액
Imp_tot=It[0:4,-1]   # 총수입

Tt_itd = Tt[0:4,4] # 총거래표의 각 산업별 및 총산업의 중간수요계
Dt_itd = Dt[0:4,4] # 국산거래표의 각 산업별 및 총산업의 중간수요계
It_itd = It[0:4,4] # 수입거래표의 각 산업별 및 총산업의 중간수요계

Tot_dmd = Tt[0:4,-4] # 각 제품별 및 전품목 총수요계
Fd =Tt[0:4,-5] # 각 제품별 최종수요
Exp = Tt[0:4,-6]  # 각 제품별 및 전품목 수출

print(Tot_sup)
print(Dom_ct)
print(Imp_tot)

print(Tt_itd)
print(Dt_itd)
print(It_itd)

print(Tot_dmd)
print(Fd)
print(Exp)

[72.0 2369.0 2552.0 4993.0]
[60.0 1792.0 2447.0 4299.0]
[12.0 577.0 105.0 694.0]
[53.0 1361.0 957.0 2371.0]
[45.0 925.0 894.0 1864.0]
[8.0 436.0 63.0 507.0]
[72.0 2369.0 2552.0 4993.0]
[19.0 1008.0 1595.0 2622.0]
[1.0 628.0 131.0 760.0]


In [47]:
Itd_r = (Tt_itd/Tot_dmd).reshape([4,1]).astype(float).round(3) # 중간수요율 = 중간수요계/총수요
Ftd_r = (Fd/Tot_dmd).reshape([4,1]).astype(float).round(3) # 최종수요율 = 최종수요계/총수요
Exp_r = (Exp/Dom_ct).reshape([4,1]).astype(float).round(3) # 수출률 = 수출/총산출액
Imp_r = (Imp_tot/Tot_sup).reshape([4,1]).astype(float).round(3) # 수입비율 = 수입/총공급

print(Itd_r)
print(Ftd_r)
print(Exp_r)
print(Imp_r)

[[0.736]
 [0.575]
 [0.375]
 [0.475]]
[[0.264]
 [0.425]
 [0.625]
 [0.525]]
[[0.017]
 [0.35 ]
 [0.054]
 [0.177]]
[[0.167]
 [0.244]
 [0.041]
 [0.139]]


In [48]:
# 중간수요율, 최종수요율, 수출률 및 수입비율 열방향으로 부치기

Dst_str = (np.concatenate((Itd_r, Ftd_r, Exp_r, Imp_r), axis=1)*100)
Dst_str_f = Dst_str.astype(float).round(1)
print(Dst_str_f)

[[73.6 26.4  1.7 16.7]
 [57.5 42.5 35.  24.4]
 [37.5 62.5  5.4  4.1]
 [47.5 52.5 17.7 13.9]]


In [49]:
Dst_f = pd.DataFrame(Dst_str_f, columns = ['중간수요율', '최종수요율', '수출률', '수입비율'],
                       index = ['농산품','공산품','서비스','전품목'])
Dst_f

,중간수요율,최종수요율,수출률,수입비율
농산품,73.6,26.4,1.7,16.7
공산품,57.5,42.5,35.0,24.4
서비스,37.5,62.5,5.4,4.1
전품목,47.5,52.5,17.7,13.9


In [50]:
# 데이터 내보내기
Input_f.to_csv('투입구조.csv', encoding="utf-8-sig")
Dst_f.to_csv('배분구조.csv', encoding="utf-8-sig")